<a href="https://colab.research.google.com/github/JonatanPolanco/Data_Quality_Testing/blob/main/PyDeequ_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Instalación de librerias**

In [1]:
!pip install pydeequ==1.0.1

In [2]:
!pip install pyspark==3.0.3

     |████████████████████████████████| 209.1 MB 45 kB/s 
     |████████████████████████████████| 198 kB 60.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.3-py2.py3-none-any.whl size=209435971 sha256=40872f8eb6d6a797876101c07a38c71454ac52186e77854ddc6ab39cabbf3b10
  Stored in directory: /root/.cache/pip/wheels/7e/6d/0a/6b0bf301bc056d9af03194b732b9f49ad2fceb205aab2984fd
Successfully built pyspark


In [ ]:
!pip install quinn

**Configuración de sesión de PySpark**

In [ ]:
from pyspark.sql import SparkSession, Row
import pydeequ
import pandas as pd

spark = (SparkSession
    .builder
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
    .getOrCreate())

Please set env variable SPARK_VERSION


**Cargando data**

In [ ]:
df = spark.sparkContext.parallelize([
            Row(a="https://www.vldb.org/pvldb/vol11/p1781-schelter.pdf", b=1, c="jobici8705@gmail"),
            Row(a="https://aws.amazon.com/blogs/big-data/test-data-quality-at-scale-with-deequ/", b=2, c="jonatan@outlook.es"),
            Row(a="https://pydeequ.readthedocs.io/_/downloads/en/latest/pdf/", b=3, c='jobici8705@')]).toDF()

**Visualizar data**

In [ ]:
df__ = df.toPandas()
df__.head()

,a,b,c
0,https://www.vldb.org/pvldb/vol11/p1781-schelte...,1,jobici8705@gmail
1,https://aws.amazon.com/blogs/big-data/test-dat...,2,jonatan@outlook.es
2,https://pydeequ.readthedocs.io/_/downloads/en/...,3,jobici8705@


**Analizadores AWS Deequ**

In [ ]:
from pydeequ.analyzers import *

analysisResult = AnalysisRunner(spark) \
                    .onData(df) \
                    .addAnalyzer(Size()) \
                    .addAnalyzer(Completeness("b")) \
                    .addAnalyzer(Completeness("c")) \
                    .run()

analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult)
analysisResult_df.show()

/usr/local/lib/python3.7/dist-packages/pydeequ/pandas_utils.py:27: UserWarning: WARNING: You passed in a Pandas DF, so we will be using our experimental utility to convert it to a PySpark DF.
  "WARNING: You passed in a Pandas DF, so we will be using our experimental utility to "


+-------+--------+------------+------------------+
| entity|instance|        name|             value|
+-------+--------+------------+------------------+
|Dataset|       *|        Size|               3.0|
| Column|       b|Completeness|               1.0|
| Column|       c|Completeness|0.6666666666666666|
+-------+--------+------------+------------------+



**Perfilamiento**

In [ ]:
from pydeequ.profiles import *

result = ColumnProfilerRunner(spark) \
    .onData(df) \
    .run()

for col, profile in result.profiles.items():
    print(profile)

StandardProfiles for column: a: {
    "completeness": 1.0,
    "approximateNumDistinctValues": 3,
    "dataType": "String",
    "isDataTypeInferred": false,
    "typeCounts": {
        "Boolean": 0,
        "Fractional": 0,
        "Integral": 0,
        "Unknown": 0,
        "String": 3
    },
    "histogram": [
        [
            "https://aws.amazon.com/blogs/big-data/test-data-quality-at-scale-with-deequ/",
            1,
            0.3333333333333333
        ],
        [
            "https://www.vldb.org/pvldb/vol11/p1781-schelter.pdf",
            1,
            0.3333333333333333
        ],
        [
            "baz",
            1,
            0.3333333333333333
        ]
    ]
}
NumericProfiles for column: b: {
    "completeness": 1.0,
    "approximateNumDistinctValues": 3,
    "dataType": "Integral",
    "isDataTypeInferred": false,
    "typeCounts": {},
    "histogram": [
        [
            "1",
            1,
            0.3333333333333333
        ],
        [
      

**Sugerencias de restricciones**

In [ ]:
from pydeequ.suggestions import *
import json

suggestionResult = ConstraintSuggestionRunner(spark) \
             .onData(df) \
             .addConstraintRule(DEFAULT()) \
             .run()

# Constraint Suggestions in JSON format 
#print(json.dumps(suggestionResult["constraint_suggestions"], indent=2))   # column_name, description, rule_description, code_for_constraint

for key, value in suggestionResult.items():
  suggestion = pd.DataFrame(value)
suggestion

,constraint_name,column_name,current_value,description,suggesting_rule,rule_description,code_for_constraint
0,"CompletenessConstraint(Completeness(b,None))",b,Completeness: 1.0,'b' is not null,CompleteIfCompleteRule(),"If a column is complete in the sample, we sugg...",".isComplete(""b"")"
1,ComplianceConstraint(Compliance('b' has no neg...,b,Minimum: 1.0,'b' has no negative values,NonNegativeNumbersRule(),If we see only non-negative numbers in a colum...,".isNonNegative(""b"")"
2,"UniquenessConstraint(Uniqueness(List(b),None))",b,ApproxDistinctness: 1.0,'b' is unique,UniqueIfApproximatelyUniqueRule(),If the ratio of approximate num distinct value...,".isUnique(""b"")"
3,"CompletenessConstraint(Completeness(a,None))",a,Completeness: 1.0,'a' is not null,CompleteIfCompleteRule(),"If a column is complete in the sample, we sugg...",".isComplete(""a"")"
4,"UniquenessConstraint(Uniqueness(List(a),None))",a,ApproxDistinctness: 1.0,'a' is unique,UniqueIfApproximatelyUniqueRule(),If the ratio of approximate num distinct value...,".isUnique(""a"")"
5,"CompletenessConstraint(Completeness(c,None))",c,Completeness: 1.0,'c' is not null,CompleteIfCompleteRule(),"If a column is complete in the sample, we sugg...",".isComplete(""c"")"
6,"UniquenessConstraint(Uniqueness(List(c),None))",c,ApproxDistinctness: 1.0,'c' is unique,UniqueIfApproximatelyUniqueRule(),If the ratio of approximate num distinct value...,".isUnique(""c"")"


**Verificación de restricciones**

In [ ]:
from pyspark.sql.types import IntegerType
from pydeequ.checks import *
from pydeequ.verification import *

check = Check(spark, CheckLevel.Warning, "Review Check")

checkResult = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(
        check.hasSize(lambda x: x >= 3) \
        .hasMin("b", lambda x: x == 0) \
        .hasDataType("b",ConstrainableDataTypes.Integral) \
        .isComplete("c")  \
        .isComplete("b")  \
        .isUnique("a")  \
        .isContainedIn("a", ["foo", "bar", "baz"]) \
        .isNonNegative("b") \
        .containsEmail("c") \
        .containsURL("a")) \
    .run()

print(f"Verification Run Status: {checkResult.status}")
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult, pandas=True)
checkResult_df

Verification Run Status: Warning


,check,check_level,check_status,constraint,constraint_status,constraint_message
0,Review Check,Warning,Warning,SizeConstraint(Size(None)),Success,
1,Review Check,Warning,Warning,"MinimumConstraint(Minimum(b,None))",Failure,Value: 1.0 does not meet the constraint requir...
2,Review Check,Warning,Warning,"AnalysisBasedConstraint(DataType(b,None),<func...",Success,
3,Review Check,Warning,Warning,"CompletenessConstraint(Completeness(c,None))",Success,
4,Review Check,Warning,Warning,"CompletenessConstraint(Completeness(b,None))",Success,
5,Review Check,Warning,Warning,"UniquenessConstraint(Uniqueness(List(a),None))",Success,
6,Review Check,Warning,Warning,ComplianceConstraint(Compliance(a contained in...,Failure,Value: 0.0 does not meet the constraint requir...
7,Review Check,Warning,Warning,ComplianceConstraint(Compliance(b is non-negat...,Success,
8,Review Check,Warning,Warning,containsEmail(c),Failure,Value: 0.3333333333333333 does not meet the co...
9,Review Check,Warning,Warning,containsURL(a),Success,



**Cargando nueva data (37 campos y 100k registros)**

In [131]:
from pyspark.sql import SparkSession, Row
import pydeequ
import pandas as pd
from pyspark import SparkContext 
from pyspark.sql.types import *

spark = (SparkSession
    .builder
    .appName("Example") \
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
    .config("spark.driver.memory", "16g")\
    .config("spark.executor.memory", "4g") \
    .getOrCreate())

**Visualizar data**

In [137]:
DF2 = pd.read_csv("100000_Records.csv") #dataframe en pandas para visualización y corrección de headers
df2= spark.read.csv("100000_Records.csv") #dataframe en spark para usar con pydeequ

In [149]:
#Se guardan los encabezados creados por spark (_c0, _c1) y encabezados con los nobres de las columnas
bad_headers = df2.schema.names
headers =list(DF2.columns)

In [139]:
DF2.head()

,Emp ID,Name Prefix,First Name,Middle Initial,Last Name,Gender,E Mail,Father's Name,Mother's Name,Mother's Maiden Name,...,SSN,Phone No.,Place Name,County,City,State,Zip,Region,User Name,Password
0,882966,Mrs.,Gwyn,E,Etzel,F,gwyn.etzel@aol.com,Sebastian Etzel,Joshua Etzel,Marden,...,640-29-9264,405-775-9884,Bromide,Johnston,Bromide,OK,74530,South,geetzel,c@sLdGgxI[sE|aJ
1,189028,Ms.,Ressie,I,Goodwyn,F,ressie.goodwyn@charter.net,Lucien Goodwyn,Sparkle Goodwyn,Lotts,...,375-37-8517,212-786-7533,Reading Center,Schuyler,Reading Center,NY,14876,Northeast,rigoodwyn,51CEPJ]r
2,479122,Mr.,Colton,B,Salzman,M,colton.salzman@hotmail.com,Dennis Salzman,Leesa Salzman,Slattery,...,073-02-5563,385-775-0225,Ibapah,Tooele,Ibapah,UT,84034,West,cbsalzman,l!Hspr@#$Dk7sq
3,484002,Ms.,Marylynn,G,Ealey,F,marylynn.ealey@cox.net,Efren Ealey,Kayleen Ealey,Gandara,...,106-98-7919,216-984-8974,Wickliffe,Lake,Wickliffe,OH,44092,Midwest,mgealey,QeJJ:vj6
4,677207,Ms.,Bula,V,Reich,F,bula.reich@gmail.com,Dominic Reich,Daphne Reich,Lister,...,753-07-4655,217-307-8597,Forrest,Livingston,Forrest,IL,61741,Midwest,bvreich,6_<OFxu\]


**Sugerencias de restricciones**

In [140]:
from pydeequ.suggestions import *

suggestionResult = ConstraintSuggestionRunner(spark) \
             .onData(df2) \
             .addConstraintRule(DEFAULT()) \
             .run()

# Constraint Suggestions in JSON format 
#print(json.dumps(suggestionResult["constraint_suggestions"], indent=2))   # column_name, description, rule_description, code_for_constraint

for key, value in suggestionResult.items():
  suggestion = pd.DataFrame(value)
  copy = suggestion ##copy of suggestion. for testing propouse

In [145]:
suggestion = suggestion.astype('string')
suggestion.head(2)

,constraint_name,column_name,current_value,description,suggesting_rule,rule_description,code_for_constraint
0,"CompletenessConstraint(Completeness(_c29,None))",_c29,Completeness: 1.0,'_c29' is not null,CompleteIfCompleteRule(),"If a column is complete in the sample, we sugg...",".isComplete(""_c29"")"
1,"CompletenessConstraint(Completeness(_c11,None))",_c11,Completeness: 1.0,'_c11' is not null,CompleteIfCompleteRule(),"If a column is complete in the sample, we sugg...",".isComplete(""_c11"")"


In [146]:
# se reemplaza los headers creados por spark con los originales
for i in range(suggestion.shape[0]):
  for j in range(len(headers)):
    if suggestion['column_name'].loc[i] == bad_headers[j]:
      suggestion.iloc[i,:] = suggestion.iloc[i,:].str.replace(bad_headers[j], headers[j])

In [157]:
suggestion.head(10) # visualizar tabla de sugerencias con headers corregidos

,constraint_name,column_name,current_value,description,suggesting_rule,rule_description,code_for_constraint
0,CompletenessConstraint(Completeness(Place Name...,Place Name,Completeness: 1.0,'Place Name' is not null,CompleteIfCompleteRule(),"If a column is complete in the sample, we sugg...",".isComplete(""Place Name"")"
1,CompletenessConstraint(Completeness(Time of Bi...,Time of Birth,Completeness: 1.0,'Time of Birth' is not null,CompleteIfCompleteRule(),"If a column is complete in the sample, we sugg...",".isComplete(""Time of Birth"")"
2,ComplianceConstraint(Compliance('Month of Join...,Month of Joining,Compliance: 1,"'Month of Joining' has value range '7', '5', '...",CategoricalRangeRule(),"If we see a categorical range for a column, we...",".isContainedIn(""Month of Joining"", [""7"", ""5"", ..."
3,CompletenessConstraint(Completeness(Month of J...,Month of Joining,Completeness: 1.0,'Month of Joining' is not null,CompleteIfCompleteRule(),"If a column is complete in the sample, we sugg...",".isComplete(""Month of Joining"")"
4,ComplianceConstraint(Compliance('Month of Join...,Month of Joining,Compliance: 0.923280767192328,"'Month of Joining' has value range '7', '5', '...",FractionalCategoricalRangeRule(0.9),If we see a categorical range for most values ...,".isContainedIn(""Month of Joining"", [""7"", ""5"", ..."
5,ComplianceConstraint(Compliance('Day of Joinin...,Day of Joining,Compliance: 1,"'Day of Joining' has value range '23', '27', '...",CategoricalRangeRule(),"If we see a categorical range for a column, we...",".isContainedIn(""Day of Joining"", [""23"", ""27"", ..."
6,CompletenessConstraint(Completeness(Day of Joi...,Day of Joining,Completeness: 1.0,'Day of Joining' is not null,CompleteIfCompleteRule(),"If a column is complete in the sample, we sugg...",".isComplete(""Day of Joining"")"
7,ComplianceConstraint(Compliance('Day of Joinin...,Day of Joining,Compliance: 0.9200407995920039,"'Day of Joining' has value range '23', '27', '...",FractionalCategoricalRangeRule(0.9),If we see a categorical range for most values ...,".isContainedIn(""Day of Joining"", [""23"", ""27"", ..."
8,ComplianceConstraint(Compliance('Weight in Kgs...,Weight in Kgs.,Compliance: 1,"'Weight in Kgs.' has value range '52', '55', '...",CategoricalRangeRule(),"If we see a categorical range for a column, we...",".isContainedIn(""Weight in Kgs."", [""52"", ""55"", ..."
9,CompletenessConstraint(Completeness(Weight in ...,Weight in Kgs.,Completeness: 1.0,'Weight in Kgs.' is not null,CompleteIfCompleteRule(),"If a column is complete in the sample, we sugg...",".isComplete(""Weight in Kgs."")"


In [ ]:
# Descargar tabla
from google.colab import files
suggestion.to_excel('suggestions.xlsx')
files.download('suggestions.xlsx')

**Verificacion de restricciones**

In [150]:
df2= spark.read.csv("100000_Records.csv", header = True) 

In [158]:
from pyspark.sql.types import IntegerType
from pydeequ.checks import *
from pydeequ.verification import *

check = Check(spark, CheckLevel.Warning, "Review Check")

checkResult = VerificationSuite(spark) \
    .onData(df2) \
    .addCheck(
      check.hasDataType("Month of Joining",ConstrainableDataTypes.Integral) \
      .isComplete("Emp ID")  \
      .isUnique("Emp ID")  \
      .isContainedIn("Gender", ["M", "F"]) \
      .isNonNegative("Salary") \
      .containsEmail("E Mail") \
      .containsSocialSecurityNumber("SSN")) \
    .run()

print(f"Verification Run Status: {checkResult.status}")
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult, pandas=True)
checkResult_df

Verification Run Status: Warning


,check,check_level,check_status,constraint,constraint_status,constraint_message
0,Review Check,Warning,Warning,AnalysisBasedConstraint(DataType(Month of Join...,Success,
1,Review Check,Warning,Warning,"CompletenessConstraint(Completeness(Emp ID,None))",Success,
2,Review Check,Warning,Warning,"UniquenessConstraint(Uniqueness(List(Emp ID),N...",Failure,Value: 0.89945 does not meet the constraint re...
3,Review Check,Warning,Warning,ComplianceConstraint(Compliance(Gender contain...,Success,
4,Review Check,Warning,Warning,ComplianceConstraint(Compliance(Salary is non-...,Success,
5,Review Check,Warning,Warning,containsEmail(E Mail),Success,
6,Review Check,Warning,Warning,containsSocialSecurityNumber(SSN),Success,


In [159]:
# Descargar tabla
from google.colab import files
checkResult_df.to_excel('verification.xlsx')
files.download('verification.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
dataF = df2.toPandas()

In [ ]:
ID = dataF['Emp ID']
print(f"Total de ID: {len(ID)}" , f"Distintos: {ID.nunique()}" )

Total de ID: 100000 Distintos: 94888
